In [15]:
import snap
import pandas as pd

In [34]:
year = 2015
#filename = 
df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

In [35]:
df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)



85786
127552


In [36]:
df_bay = df_bay[['tabblk2010', 'ctyname']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode"})

df_bay_outer = df_bay_outer[['tabblk2010', 'ctyname']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode"})

print len(df.index)

result = pd.merge(df, df_bay, on='w_geocode')
print len(result.index)
result2 = pd.merge(result, df_bay_outer, on='h_geocode')
print len(result2.index)

13417793
2819148
2486755


In [ ]:

for index, row in result2.iterrows():
    if row['S000'] == 0:
        continue
    if row['h_geocode'] in census_block_outdeg_dict.keys():
        census_block_outdeg_dict[row['h_geocode']] += row['S000']
    else:
        census_block_outdeg_dict[row['h_geocode']] = row['S000']
    if index % 100000 == 0:
        print index

In [28]:
result3 = result2[['h_geocode', 'w_geocode']]
result3.head()
result3.to_csv('edgelist'+str(year)+'_unweighted.txt', header=None, index=None, sep=' ', mode='a')

In [29]:
G = snap.LoadEdgeList(snap.PNGraph, 'edgelist'+str(year)+'_unweighted.txt', 0, 1, ' ')

In [15]:
import sys
print sys.version

2.7.10 (default, Oct  6 2017, 22:29:07) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.31)]


In [30]:
G.GetNodes()

90485

In [31]:
G.GetEdges()


2121308

In [23]:
print snap.CntSelfEdges(G)

1755


In [37]:
print result2['S000'].sum()

2704262


In [7]:
CfVec = snap.TFltPrV()
Cf = snap.GetClustCf(G, CfVec, 1)
print Cf

RuntimeError: Execution stopped: (0<=ValN)&&(ValN<Vals) [Reason:'Index:-266585020 Vals:2146567384 MxVals:2146567384 Type:4TVecIS_I4TIntiEiE'], file ../../snap/glib-core/ds.h, line 491

In [9]:
G = snap.LoadEdgeList(snap.PNGraph, 'test.txt', 0, 1, ' ')
print G.GetNodes()
print G.GetEdges()

405
500


In [11]:
Cf = snap.GetClustCf(G, 1)
print Cf

RuntimeError: Execution stopped: (0<=ValN)&&(ValN<Vals) [Reason:'Index:-347110296 Vals:2140576379 MxVals:2140576379 Type:4TVecIS_I4TIntiEiE'], file ../../snap/glib-core/ds.h, line 491

In [12]:
print snap.CntSelfEdges(G)

1


In [9]:
import snap
import pandas as pd

df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#filename = 

df_bay = df_bay[['tabblk2010', 'trct']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct"})

df_bay_outer = df_bay_outer[['tabblk2010', 'trct']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode", "trct": "h_trct"})

for year in years:
    df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

    df_work_bay = pd.merge(df, df_bay, on='w_geocode')

    df_work_bay_home_bayouter = pd.merge(df_work_bay, df_bay_outer, on='h_geocode')
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_geocode', 'h_geocode', 'w_trct', 'h_trct', 'S000']]
    
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'S000']]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum().reset_index()
    
    # new
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='w_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"w_lat", "lon":"w_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"w_trct":"h_trct"})
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='h_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"h_lat", "lon":"h_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"h_trct":"w_trct"})
    
    dist = []
    w_lat = df_work_bay_home_bayouter['w_lat']
    w_lon = df_work_bay_home_bayouter['w_lon']
    h_lat = df_work_bay_home_bayouter['h_lat']
    h_lon = df_work_bay_home_bayouter['h_lon']
    
    for i in range(len(w_lat)):
        coords_1 = (w_lat[i], w_lon[i])
        coords_2 = (h_lat[i], h_lon[i])
        dist.append(geopy.distance.distance(coords_1, coords_2).miles) 
        
    df_work_bay_home_bayouter['dist'] = dist
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.drop(columns=['w_lat','w_lon','h_lat','h_lon'])
    
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_'+str(year)+'.csv',index=False)
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_'+str(year)+'.txt',index=False, header=False, sep=' ')
    
    census_tract_indeg_dict = {}

    for index, row in df_work_bay_home_bayouter.iterrows():
        if row['w_trct'] in census_tract_indeg_dict.keys():
            census_tract_indeg_dict[row['w_trct']] += row['S000']
        else:
            census_tract_indeg_dict[row['w_trct']] = row['S000']

            
    indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
    indeg_df['w_trct'] = indeg_df.index
    indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

    indeg_df.to_csv('bay_outer_tract_indeg_'+str(year)+'.csv',index=False)



85786
127552


In [5]:
# examine only poor people

import snap
import pandas as pd
import geopy.distance

df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


#years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#filename = blklatdd blklondd

df_bay = df_bay[['tabblk2010', 'trct']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct"})

df_bay_outer = df_bay_outer[['tabblk2010', 'trct']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode", "trct": "h_trct"})
df_lat_lon = pd.read_csv('tract_lat_lon.csv')

for year in years:
    df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

    df_work_bay = pd.merge(df, df_bay, on='w_geocode')

    df_work_bay_home_bayouter = pd.merge(df_work_bay, df_bay_outer, on='h_geocode')
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_geocode', 'h_geocode', 'w_trct', 'h_trct', 'SE01', 'SE02']]
    df_work_bay_home_bayouter['low_income'] = df_work_bay_home_bayouter['SE01'] + df_work_bay_home_bayouter['SE02']
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'low_income']]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[df_work_bay_home_bayouter.low_income != 0]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum().reset_index()
    
    # new
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='w_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"w_lat", "lon":"w_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"w_trct":"h_trct"})
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='h_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"h_lat", "lon":"h_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"h_trct":"w_trct"})
    
    dist = []
    w_lat = df_work_bay_home_bayouter['w_lat']
    w_lon = df_work_bay_home_bayouter['w_lon']
    h_lat = df_work_bay_home_bayouter['h_lat']
    h_lon = df_work_bay_home_bayouter['h_lon']
    for i in range(len(w_lat)):
        coords_1 = (w_lat[i], w_lon[i])
        coords_2 = (h_lat[i], h_lon[i])
        dist.append(geopy.distance.distance(coords_1, coords_2).miles) 
        
    df_work_bay_home_bayouter['dist'] = dist
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.drop(columns=['w_lat','w_lon','h_lat','h_lon'])
    
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_'+str(year)+'_poor.csv',index=False)
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_'+str(year)+'_poor.txt',index=False, header=False, sep=' ')
    
    census_tract_indeg_dict = {}

    for index, row in df_work_bay_home_bayouter.iterrows():
        if row['w_trct'] in census_tract_indeg_dict.keys():
            census_tract_indeg_dict[row['w_trct']] += row['low_income']
        else:
            census_tract_indeg_dict[row['w_trct']] = row['low_income']
#         if index % 100000 == 0:
#             print index
            
    indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
    indeg_df['w_trct'] = indeg_df.index
    indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

    indeg_df.to_csv('bay_outer_tract_indeg_'+str(year)+'_poor.csv',index=False)



85786
127552


In [6]:
# examine only rich people

import snap
import pandas as pd
import geopy.distance

df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#filename = blklatdd blklondd

df_bay = df_bay[['tabblk2010', 'trct']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct"})

df_bay_outer = df_bay_outer[['tabblk2010', 'trct']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode", "trct": "h_trct"})
df_lat_lon = pd.read_csv('tract_lat_lon.csv')

for year in years:
    df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

    df_work_bay = pd.merge(df, df_bay, on='w_geocode')

    df_work_bay_home_bayouter = pd.merge(df_work_bay, df_bay_outer, on='h_geocode')
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_geocode', 'h_geocode', 'w_trct', 'h_trct', 'SE03']]
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'SE03']]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[df_work_bay_home_bayouter.SE03 != 0]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum().reset_index()
    
    # new
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='w_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"w_lat", "lon":"w_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"w_trct":"h_trct"})
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='h_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"h_lat", "lon":"h_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"h_trct":"w_trct"})
    
    dist = []
    w_lat = df_work_bay_home_bayouter['w_lat']
    w_lon = df_work_bay_home_bayouter['w_lon']
    h_lat = df_work_bay_home_bayouter['h_lat']
    h_lon = df_work_bay_home_bayouter['h_lon']
    for i in range(len(w_lat)):
        coords_1 = (w_lat[i], w_lon[i])
        coords_2 = (h_lat[i], h_lon[i])
        dist.append(geopy.distance.distance(coords_1, coords_2).miles) 
        
    df_work_bay_home_bayouter['dist'] = dist
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.drop(columns=['w_lat','w_lon','h_lat','h_lon'])
    
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_rich_'+str(year)+'.csv',index=False)
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_rich_'+str(year)+'.txt',index=False, header=False, sep=' ')
    
    census_tract_indeg_dict = {}

    for index, row in df_work_bay_home_bayouter.iterrows():
        if row['w_trct'] in census_tract_indeg_dict.keys():
            census_tract_indeg_dict[row['w_trct']] += row['SE03']
        else:
            census_tract_indeg_dict[row['w_trct']] = row['SE03']
#         if index % 100000 == 0:
#             print index
            
    indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
    indeg_df['w_trct'] = indeg_df.index
    indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

    indeg_df.to_csv('bay_outer_tract_indeg_rich_'+str(year)+'.csv',index=False)



85786
127552


In [11]:
# examine only workers under 30

import snap
import pandas as pd
import geopy.distance

df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#filename = blklatdd blklondd

df_bay = df_bay[['tabblk2010', 'trct']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct"})

df_bay_outer = df_bay_outer[['tabblk2010', 'trct']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode", "trct": "h_trct"})
df_lat_lon = pd.read_csv('tract_lat_lon.csv')

for year in years:
    df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

    df_work_bay = pd.merge(df, df_bay, on='w_geocode')

    df_work_bay_home_bayouter = pd.merge(df_work_bay, df_bay_outer, on='h_geocode')
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_geocode', 'h_geocode', 'w_trct', 'h_trct', 'SA01']]
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'SA01']]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[df_work_bay_home_bayouter.SA01 != 0]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum().reset_index()
    
    # new
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='w_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"w_lat", "lon":"w_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"w_trct":"h_trct"})
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='h_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"h_lat", "lon":"h_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"h_trct":"w_trct"})
    
    dist = []
    w_lat = df_work_bay_home_bayouter['w_lat']
    w_lon = df_work_bay_home_bayouter['w_lon']
    h_lat = df_work_bay_home_bayouter['h_lat']
    h_lon = df_work_bay_home_bayouter['h_lon']
    for i in range(len(w_lat)):
        coords_1 = (w_lat[i], w_lon[i])
        coords_2 = (h_lat[i], h_lon[i])
        dist.append(geopy.distance.distance(coords_1, coords_2).miles) 
        
    df_work_bay_home_bayouter['dist'] = dist
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.drop(columns=['w_lat','w_lon','h_lat','h_lon'])
    
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_young_'+str(year)+'.csv',index=False)
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_young_'+str(year)+'.txt',index=False, header=False, sep=' ')
    
    continue
    census_tract_indeg_dict = {}

    for index, row in df_work_bay_home_bayouter.iterrows():
        if row['w_trct'] in census_tract_indeg_dict.keys():
            census_tract_indeg_dict[row['w_trct']] += row['SA01']
        else:
            census_tract_indeg_dict[row['w_trct']] = row['SA01']
#         if index % 100000 == 0:
#             print index
            
    indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
    indeg_df['w_trct'] = indeg_df.index
    indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

    indeg_df.to_csv('bay_outer_tract_indeg_rich_'+str(year)+'.csv',index=False)

85786
127552


In [12]:
# examine only workers over 55

import snap
import pandas as pd
import geopy.distance

df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#filename = blklatdd blklondd

df_bay = df_bay[['tabblk2010', 'trct']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct"})

df_bay_outer = df_bay_outer[['tabblk2010', 'trct']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode", "trct": "h_trct"})
df_lat_lon = pd.read_csv('tract_lat_lon.csv')

for year in years:
    df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

    df_work_bay = pd.merge(df, df_bay, on='w_geocode')

    df_work_bay_home_bayouter = pd.merge(df_work_bay, df_bay_outer, on='h_geocode')
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_geocode', 'h_geocode', 'w_trct', 'h_trct', 'SA03']]
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'SA03']]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[df_work_bay_home_bayouter.SA03 != 0]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum().reset_index()
    
    # new
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='w_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"w_lat", "lon":"w_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"w_trct":"h_trct"})
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='h_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"h_lat", "lon":"h_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"h_trct":"w_trct"})
    
    dist = []
    w_lat = df_work_bay_home_bayouter['w_lat']
    w_lon = df_work_bay_home_bayouter['w_lon']
    h_lat = df_work_bay_home_bayouter['h_lat']
    h_lon = df_work_bay_home_bayouter['h_lon']
    for i in range(len(w_lat)):
        coords_1 = (w_lat[i], w_lon[i])
        coords_2 = (h_lat[i], h_lon[i])
        dist.append(geopy.distance.distance(coords_1, coords_2).miles) 
        
    df_work_bay_home_bayouter['dist'] = dist
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.drop(columns=['w_lat','w_lon','h_lat','h_lon'])
    
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_old_'+str(year)+'.csv',index=False)
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_old_'+str(year)+'.txt',index=False, header=False, sep=' ')
    
    continue
    census_tract_indeg_dict = {}

    for index, row in df_work_bay_home_bayouter.iterrows():
        if row['w_trct'] in census_tract_indeg_dict.keys():
            census_tract_indeg_dict[row['w_trct']] += row['SA03']
        else:
            census_tract_indeg_dict[row['w_trct']] = row['SA03']
#         if index % 100000 == 0:
#             print index
            
    indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
    indeg_df['w_trct'] = indeg_df.index
    indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

    indeg_df.to_csv('bay_outer_tract_indeg_rich_'+str(year)+'.csv',index=False)

85786
127552


In [13]:
# examine only women

import snap
import pandas as pd
import geopy.distance

df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#filename = blklatdd blklondd

df_bay = df_bay[['tabblk2010', 'trct']]
df_bay = df_bay.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct"})

df_bay_outer = df_bay_outer[['tabblk2010', 'trct']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "h_geocode", "trct": "h_trct"})
df_lat_lon = pd.read_csv('tract_lat_lon.csv')

for year in years:
    df = pd.read_csv("./ca_od_main_JT01_"+str(year)+".csv")

    df_work_bay = pd.merge(df, df_bay, on='w_geocode')

    df_work_bay_home_bayouter = pd.merge(df_work_bay, df_bay_outer, on='h_geocode')
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_geocode', 'h_geocode', 'w_trct', 'h_trct', 'CS02']]
    
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'CS02']]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter[df_work_bay_home_bayouter.CS02 != 0]
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum().reset_index()
    
    # new
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='w_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"w_lat", "lon":"w_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"w_trct":"h_trct"})
    df_work_bay_home_bayouter = pd.merge(df_work_bay_home_bayouter, df_lat_lon, on='h_trct')
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.rename(index=str, columns={"lat":"h_lat", "lon":"h_lon"})
    df_lat_lon = df_lat_lon.rename(index=str, columns={"h_trct":"w_trct"})
    
    dist = []
    w_lat = df_work_bay_home_bayouter['w_lat']
    w_lon = df_work_bay_home_bayouter['w_lon']
    h_lat = df_work_bay_home_bayouter['h_lat']
    h_lon = df_work_bay_home_bayouter['h_lon']
    for i in range(len(w_lat)):
        coords_1 = (w_lat[i], w_lon[i])
        coords_2 = (h_lat[i], h_lon[i])
        dist.append(geopy.distance.distance(coords_1, coords_2).miles) 
        
    df_work_bay_home_bayouter['dist'] = dist
    df_work_bay_home_bayouter = df_work_bay_home_bayouter.drop(columns=['w_lat','w_lon','h_lat','h_lon'])
    
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_women_'+str(year)+'.csv',index=False)
    df_work_bay_home_bayouter.to_csv('bay_outer_tract_women_'+str(year)+'.txt',index=False, header=False, sep=' ')
    
    continue
    census_tract_indeg_dict = {}

    for index, row in df_work_bay_home_bayouter.iterrows():
        if row['w_trct'] in census_tract_indeg_dict.keys():
            census_tract_indeg_dict[row['w_trct']] += row['CS02']
        else:
            census_tract_indeg_dict[row['w_trct']] = row['CS02']
#         if index % 100000 == 0:
#             print index
            
    indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
    indeg_df['w_trct'] = indeg_df.index
    indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

    indeg_df.to_csv('bay_outer_tract_indeg_women_'+str(year)+'.csv',index=False)

85786
127552


KeyError: "['CS02'] not in index"

In [13]:
year = 2010
indeg_df = pd.DataFrame.from_dict(census_tract_indeg_dict, orient='index')
indeg_df['w_trct'] = indeg_df.index
indeg_df = indeg_df.rename(index=str, columns={0: "indeg"})

indeg_df.to_csv('bay_outer_tract_indeg_'+str(year)+'.csv',index=False)

In [4]:
print len(df_work_bay_home_bayouter)
print df_work_bay_home_bayouter['S000'].sum()

2121308
2284949


In [5]:
df_g = df_work_bay_home_bayouter[['w_trct', 'h_trct', 'S000']]
df_g = df_work_bay_home_bayouter.groupby(['w_trct', 'h_trct']).sum()

In [7]:
print len(df_g)
print df_g['S000'].sum()

561874
2284949


In [5]:
df_xwalk = pd.read_csv('./ca_xwalk.csv')
bay_counties = ['San Francisco County, CA', 'Marin County, CA', 'Alameda County, CA', 'Santa Clara County, CA', 'Contra Costa County, CA', 'San Mateo County, CA']
outer_counties = bay_counties + ['Sonoma County, CA', 'San Joaquin County, CA', 'Napa County, CA', 'Santa Cruz County, CA', 'Solano County, CA']
df_bay_outer = df_xwalk.loc[df_xwalk['ctyname'].isin(outer_counties)]
df_bay = df_xwalk.loc[df_xwalk['ctyname'].isin(bay_counties)]
print len(df_bay.index)
print len(df_bay_outer.index)


years = [2010, 2011, 2012, 2013, 2014, 2015]
#filename = blklatdd blklondd

df_bay_outer = df_bay_outer[['tabblk2010', 'trct', 'blklatdd', 'blklondd']]
df_bay_outer = df_bay_outer.rename(index=str, columns={"tabblk2010": "w_geocode", "trct" : "w_trct", "blklatdd": "lat", "blklondd": "lon"})

df_bay_outer = df_bay_outer.drop(columns=['w_geocode'])
df_bay_outer = df_bay_outer.groupby(['w_trct']).mean()
df_bay_outer.to_csv("tract_lat_lon.csv")
print len(df_bay_outer)

85786
127552
1780
